# Particle Swarm Optimization

### (using functions)

In [15]:
import pyswarm as ps

In [13]:
#particle swarm optimization function
def ps_o(fitness, lb, ub, swarm_size, max_it, dimensions):
    x = []
    x_pbest = []
    x_gbest = []
    vel = []
    
    #assign random values as initial position of the particles
    for i in range(swarm_size):
        x.append([])
        x_pbest.append([])
        for j in range(dimensions):
            x[i].append(random.uniform(lb[j], ub[j]+1))
            x_pbest[i].append[x[i]]
    
    #find initial group best of the population
    for i in range(swarm_size):
        if i == 1:
            x_gbest = x[i]
        elif fitness(x[i]) < fitness(x_gbest[i])
            x_gbest[i] = x[i]
    
    #assigning v_min and v_max
    vel_min = []
    vel_max = []
    for i in range(dimensions):
        vel_min[i] = -(ub[i] - lb[i])/2
        vel_max[i] = (ub[i] - lb[i])/2
        
    #assigning random values in the range (v_min[i], v_max[i]) as initial velocities of the particles
    for i in range(swarm_size):
        vel.append()
        for j in range(dimensions):
            vel[i].append(random.randint(vel_min[j], vel_max[j]))
            
    #c1 and c2 are spring constants are set to 1.494
    #r1 and r2 are random numbers between 0 and 1
    r1 = random.uniform(0,1)
    r2 = random.uniform(0,1)
            
    #optimization function - main task of optimizing the position & velocities of the particles
    for iter in range(max_it):
        for i in range(swarm_size):
            for j in range(dimensions):
                #finding velocity of the particle
                vel[i][j] = weight*v[i][j] + r1*c1(x_pbest[i][j] - x[i][j]) + r2*c2(x_gbest[j] - x[i][j])
                
                #checking the limit of velocity, if goes beyond then sack it
                if v[i][j] > v_max:
                    vel[i][j] = (vel_max / abs(vel[i][j])*vel[i][j]
                
                #finding the position of the particles
                x[i][j] = x[i][j] + v[i][j]
                
                #checking constraints on the position
                if x[i][j] < lb[j]:
                    x[i][j] = lb[j]
                if x[i][j] > ub[j]:
                    x[i][j] = ub[j]
                
                if fitness(x[i]) < fitness(x_pbest[i]):
                    x_pbest[i] = x[i]
                if fitness(x[i]) < fitness(x_gbest)
                    x_gbest = x[i]
                
    return x_gbest, fitness(x_gbest)